In [ ]:
import numpy as np
import cv2 as cv2
import tensorflow as tf

from random import shuffle

from tqdm import tqdm
import os

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report

Pre-Processing IMG

In [175]:
#travel to dir of image
width = 224
num_class = 2

trainpath = 'modenessTest/labelData/'
#testpath = 'Mold/TrainMold/'


trainImg = [trainpath + f for f in os.listdir(trainpath)]
#testImg = [testpath + f for f in os.listdir(testpath)]

shuffle(trainImg)

print(trainImg)

['Mold/TrainMold/Bad(4).JPG', 'Mold/TrainMold/Ok(435).JPG', 'Mold/TrainMold/Ok(414).JPG', 'Mold/TrainMold/Ok(31).JPG', 'Mold/TrainMold/Ok(101).JPG', 'Mold/TrainMold/Bad(421).JPG', 'Mold/TrainMold/Ok(125).JPG', 'Mold/TrainMold/Bad(191).JPG', 'Mold/TrainMold/Bad(353).JPG', 'Mold/TrainMold/Bad(430).JPG', 'Mold/TrainMold/Ok(398).JPG', 'Mold/TrainMold/Bad(329).JPG', 'Mold/TrainMold/Ok(167).JPG', 'Mold/TrainMold/Bad(16).JPG', 'Mold/TrainMold/Bad(319).JPG', 'Mold/TrainMold/Ok(237).JPG', 'Mold/TrainMold/Ok(401).JPG', 'Mold/TrainMold/Ok(315).JPG', 'Mold/TrainMold/Ok(213).JPG', 'Mold/TrainMold/Bad(460).JPG', 'Mold/TrainMold/Bad(227).JPG', 'Mold/TrainMold/Bad(99).JPG', 'Mold/TrainMold/Bad(452).JPG', 'Mold/TrainMold/Ok(231).JPG', 'Mold/TrainMold/Bad(365).JPG', 'Mold/TrainMold/Bad(256).JPG', 'Mold/TrainMold/Bad(243).JPG', 'Mold/TrainMold/Ok(127).JPG', 'Mold/TrainMold/Bad(328).JPG', 'Mold/TrainMold/Ok(276).JPG', 'Mold/TrainMold/Bad(69).JPG', 'Mold/TrainMold/Ok(412).JPG', 'Mold/TrainMold/Ok(349).JPG'

In [176]:
#to label to answer of each img (function)
def getlabel(imgName):
   l = imgName.split('/')[2]
   l = l.split("(")[0]
   #
   if(l == 'Ok'):
     return(np.array([1,0]))
   elif(l == 'Bad'):
     return(np.array([0,1]))


In [177]:
#read img file and append in array (function)
def img2data(path):
  trainData = []
  for item in tqdm((path)):
      img = cv2.imread(item)
      img = cv2.resize(img, (width,width) )

      trainData.append([np.array(img), getlabel(item)])

  return trainData

#read img file and append in array
trainSet = img2data(trainImg)

In [179]:
#Split Test and Train dataset
train = trainSet[:-500]
test = trainSet[-500:]


xtrain = np.array([i[0] for i in train]).reshape(-1, width, width, 3)/255
ytrain = np.array([i[1] for i in train])

xtest = np.array([i[0] for i in test]).reshape(-1, width, width, 3)/255
ytest = np.array([i[1] for i in test])

620
300


*End PreProcess IMG*

*MobileNetV2*

In [180]:
baseMobileNetV2Model = tf.keras.applications.MobileNetV2(
   input_shape = (224,224,3), 
   include_top = False, 
   weights="imagenet")
baseMobileNetV2Model.trainable = False

avgPoolingLayer =  tf.keras.layers.GlobalAveragePooling2D()(baseMobileNetV2Model.output)
predLayer = tf.keras.layers.Dense(units=2, activation='softmax')(avgPoolingLayer)

MobileNetV2model = tf.keras.models.Model(inputs = baseMobileNetV2Model.input, outputs = predLayer)

MobileNetV2model.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics = ["accuracy"])
MobileNetV2model.fit(xtrain, ytrain, epochs=100, validation_data = (xtest,ytest), batch_size = 100)

In [182]:
pred = []
for img in xtest:
   rimg = np.reshape(img, (-1,224,224,3))
   result = np.argmax(MobileNetV2model.predict(rimg))
   if result == 0:
      pred.append([1,0])
   if result == 1:
      pred.append([0,1])


print(classification_report(pred, ytest))



1/1 [==============================] - 0s 385ms/step
              precision    recall  f1-score   support

           0       0.63      0.74      0.68       131
           1       0.77      0.66      0.71       169

   micro avg       0.70      0.70      0.70       300
   macro avg       0.70      0.70      0.70       300
weighted avg       0.71      0.70      0.70       300
 samples avg       0.70      0.70      0.70       300



*Finished*

DenseNet

In [183]:
baseDenseModel = tf.keras.applications.DenseNet121(
   input_shape = (224,224,3),
   include_top=False,
   weights="imagenet",
)

baseDenseModel.trainable = False
avgPoolingLayerDen =  tf.keras.layers.GlobalAveragePooling2D()(baseDenseModel.output)
predLayerDen = tf.keras.layers.Dense(units=2, activation='softmax')(avgPoolingLayerDen)

DenseNetmodel = tf.keras.models.Model(inputs = baseDenseModel.input, outputs = predLayerDen)
DenseNetmodel.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics = ["accuracy"])
DenseNetmodel.fit(xtrain, ytrain, epochs=100 , validation_data = (xtest,ytest), batch_size = 100)

In [186]:
predDen = []
for img in xtest:
   rimg = np.reshape(img, (-1,224,224,3))
   result = np.argmax(DenseNetmodel.predict(rimg))
   if result == 0:
      predDen.append([1,0])
   if result == 1:
      predDen.append([0,1])

print(classification_report(predDen, ytest))

1/1 [==============================] - 1s 1s/step
              precision    recall  f1-score   support

           0       0.64      0.73      0.68       135
           1       0.75      0.66      0.70       165

   micro avg       0.69      0.69      0.69       300
   macro avg       0.69      0.69      0.69       300
weighted avg       0.70      0.69      0.69       300
 samples avg       0.69      0.69      0.69       300



*Finished*

InceptionV3

In [187]:
baseInceptionModel = tf.keras.applications.InceptionV3(
   input_shape = (224,224,3),
   include_top=False,
   weights="imagenet",
)

baseInceptionModel.trainable = False
avgPoolingLayerIn =  tf.keras.layers.GlobalAveragePooling2D()(baseInceptionModel.output)
predLayerIn = tf.keras.layers.Dense(units=2, activation='softmax')(avgPoolingLayerIn)

Inceptionmodel = tf.keras.models.Model(inputs = baseInceptionModel.input, outputs = predLayerIn)
Inceptionmodel.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics = ["accuracy"])
Inceptionmodel.fit(xtrain, ytrain, epochs=100 , validation_data = (xtest,ytest), batch_size = 100)

In [ ]:
predIn = []
for img in xtest:
   rimg = np.reshape(img, (-1,224,224,3))
   result = np.argmax(Inceptionmodel.predict(rimg))
   if result == 0:
      predIn.append([1,0])
   if result == 1:
      predIn.append([0,1])

print(classification_report(predIn, ytest))

*Finished*

Xception

In [191]:
baseXceptionmodel = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    classifier_activation="softmax",
)

baseXceptionmodel.trainable = False
avgPoolingLayer3 =  tf.keras.layers.GlobalAveragePooling2D()(baseXceptionmodel.output)
predLayer3 = tf.keras.layers.Dense(units=2, activation='softmax')(avgPoolingLayer3)

Xceptionmodel = tf.keras.models.Model(inputs = baseXceptionmodel.input, outputs = predLayer3)
Xceptionmodel.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics = ["accuracy"])
Xceptionmodel.fit(xtrain, ytrain, epochs=100, validation_data = (xtest,ytest), batch_size = 100)

In [ ]:
pred3 = []
for img in xtest:
   rimg = np.reshape(img, (-1,224,224,3))
   result = np.argmax(Xceptionmodel.predict(rimg))
   if result == 0:
      pred3.append([1,0])
   if result == 1:
      pred3.append([0,1])
   

print(classification_report(pred3, ytest))

*Finished*

Summary all

In [195]:
print("MobileNetV2 report")
print(classification_report(pred, ytest))
print("########################################")

print("DenseNet121 report")
print(classification_report(predDen, ytest))
print("########################################")

print("Incception report")
print(classification_report(predIn, ytest))
print("########################################")

print("Xception report")
print(classification_report(pred3, ytest))
print("########################################")

MobileNetV2 report
              precision    recall  f1-score   support

           0       0.63      0.74      0.68       131
           1       0.77      0.66      0.71       169

   micro avg       0.70      0.70      0.70       300
   macro avg       0.70      0.70      0.70       300
weighted avg       0.71      0.70      0.70       300
 samples avg       0.70      0.70      0.70       300

########################################
DenseNet121 report
              precision    recall  f1-score   support

           0       0.64      0.73      0.68       135
           1       0.75      0.66      0.70       165

   micro avg       0.69      0.69      0.69       300
   macro avg       0.69      0.69      0.69       300
weighted avg       0.70      0.69      0.69       300
 samples avg       0.69      0.69      0.69       300

########################################
Incception report
              precision    recall  f1-score   support

           0       0.90      0.63      0.74  